# Detecting kranocyte populations in the datasets

In [ ]:
import scanpy as sc
import scanpy.external as sce
import pandas as pd
import numpy as np
import os
import triku as tk
import matplotlib.pyplot as plt
import matplotlib as mpl
from tqdm import tqdm
import ray
import subprocess
from scipy.sparse import csr_matrix
from IPython.display import display, HTML

from bokeh.io import show, output_notebook, reset_output

reset_output()
output_notebook()

In [ ]:
from cellassign import assign_cats

help(assign_cats)

In [ ]:
# Palettes for UMAP gene expression

magma = [plt.get_cmap('magma')(i) for i in np.linspace(0,1, 80)]
magma[0] = (0.88, 0.88, 0.88, 1)
magma = mpl.colors.LinearSegmentedColormap.from_list("", magma[:65])

seed = 0

In [ ]:
mpl.rcParams['figure.dpi'] = 150

In [ ]:
A_markers = ['6030408B16Rik', 'Smim41', 'Col9a2', 'Dlk1', 'Shisa3',  'Saa1',  'Nipal1']
A_markers_extra = ['Kcnk2',  # Not specific enough
                   'Adamtsl2',  # Not specific enough
                   'Cst6',  # Teno marker
                   'Sorcs2',  # Not specific enough
                   'Susd5',  # Not specific enough
                   'Rgs17',  # Not specific enough
                   'Gfra2']  # Marks immune population
B_markers = ['Lypd2', 'Wnt6', 'Cldn1', 'Moxd1', 'Mansc4', 'Dleu7', 'Efnb3', 'Stra6', 'Sbspon', 'Ace2', 'Hcn4', 'Cldn22', 'Wnt10a', 'Ocln']  
B_markers_extra = ['Itgb4',  # Expressed by immune
                   'Ralgps2',  # Expressed by immune
                   'Ctxn3',  # Myonuclei
                   'Tenm2',  # Krano A
                   'Sfrp5',  # Glial 
                   'Perp',  # Glial + peris
                   'Krt19',  # APC
                   'Gpm6a',  # Immune
                   'Ctxn3',  # Tenocytes
                   'Crabp2',  # Pericytes / teno
                   'Slc6a13',  # APC
                   'Itga7']  # Immune

In [ ]:
list_colors = ['#bcbcbc', '#900C3F', '#286e87']

## Oprescu

In [ ]:
adata_oprescu_d0 = sc.read('data/processed/oprescu_d0.h5')

# These are used only for a figure
adata_oprescu_d05 = sc.read('data/processed/oprescu_d05.h5')
adata_oprescu_d2 = sc.read('data/processed/oprescu_d2.h5')
adata_oprescu_d35 = sc.read('data/processed/oprescu_d35.h5')
adata_oprescu_d5 = sc.read('data/processed/oprescu_d5.h5')
adata_oprescu_d10 = sc.read('data/processed/oprescu_d10.h5')
adata_oprescu_d21 = sc.read('data/processed/oprescu_d21.h5')

In [ ]:
adata_oprescu_d0.obs['id'] = adata_oprescu_d0.obs_names
assign_cats(adata_oprescu_d0, column_groupby='id', dict_cats={'A': A_markers, 'B': B_markers}, min_score=0.6, 
            others_name='-', key_added='krano_type')
adata_oprescu_d0.uns['krano_type_colors'] = list_colors

In [ ]:
sc.pl.umap(adata_oprescu_d0, color=['krano_type', 'cell_type'] , cmap=magma, alpha=0.6)

In [ ]:
sc.pl.umap(adata_oprescu_d0, color=['krano_type', 'cell_type', 'Mpzl2'] , cmap=magma, alpha=0.6)

In [ ]:
sc.pl.umap(adata_oprescu_d0, color=['krano_type', 'cell_type'] + [i for i in A_markers if i in adata_oprescu_d0.var_names], 
           cmap=magma, alpha=0.6)

In [ ]:
sc.pl.umap(adata_oprescu_d0, color=['krano_type', 'cell_type'] + [i for i in B_markers if i in adata_oprescu_d0.var_names], 
           cmap=magma, alpha=0.6)

## Subsample of populations

In [ ]:
adata_oprescu_d0_sub = adata_oprescu_d0[adata_oprescu_d0.obs['cell_type'].isin(['FAP Lum+', 'FAP Prg4+', 'Endothelial', 'Perivascular', 'Tenocyte', 'Glial cell', 'Neural cell'])]

sc.pp.filter_genes(adata_oprescu_d0_sub, min_counts=1)

sc.pp.pca(adata_oprescu_d0_sub, random_state=seed, n_comps=30)
sc.pp.neighbors(adata_oprescu_d0_sub, random_state=seed, n_neighbors=int(len(adata_oprescu_d0_sub) ** 0.5 // 4), metric='cosine')
tk.tl.triku(adata_oprescu_d0_sub)

sc.tl.umap(adata_oprescu_d0_sub, min_dist=0.7, random_state=seed)
sc.tl.leiden(adata_oprescu_d0_sub, resolution=3, random_state=seed)

assign_cats(adata_oprescu_d0_sub, column_groupby='id', dict_cats={'A': A_markers, 'B': B_markers}, min_score=0.7, others_name='-', key_added='krano_type')

sc.pl.umap(adata_oprescu_d0_sub, color=['leiden', 'n_counts', 'cell_type', 'krano_type'], legend_loc='on data')

In [ ]:
adata_oprescu_d0_sub.write_h5ad('data/processed/oprescu_d0_sub.h5')
adata_oprescu_d0.write_h5ad('data/processed/oprescu_d0.h5')

## Scott

In [ ]:
adata_scott_d0 = sc.read('data/processed/scott_d0.h5')

In [ ]:
adata_scott_d0.obs['id'] = adata_scott_d0.obs_names
assign_cats(adata_scott_d0, column_groupby='id', dict_cats={'A': A_markers, 'B': B_markers}, min_score=0.4, 
            others_name='-', key_added='krano_type')
adata_scott_d0.uns['krano_type_colors'] = list_colors

In [ ]:
sc.pl.umap(adata_scott_d0, color=['krano_type', 'cell_type'] , cmap=magma, alpha=0.6)

In [ ]:
sc.pl.umap(adata_scott_d0, color=['krano_type', 'cell_type'] + [i for i in A_markers if i in adata_scott_d0.var_names], 
           cmap=magma, alpha=0.6)

In [ ]:
sc.pl.umap(adata_scott_d0, color=['krano_type', 'cell_type'] + [i for i in B_markers if i in adata_scott_d0.var_names], 
           cmap=magma, alpha=0.6)

## Subsample of populations

In [ ]:
adata_scott_d0_sub = adata_scott_d0[adata_scott_d0.obs['cell_type'].isin(['FAP Lum+', 'FAP Prg4+', 'Endothelial', 'Perivascular', 'Tenocyte', 'Neural cell', 'Glial cell'])]

sc.pp.filter_genes(adata_scott_d0_sub, min_counts=1)

sc.pp.pca(adata_scott_d0_sub, random_state=seed, n_comps=30)
sc.pp.neighbors(adata_scott_d0_sub, random_state=seed, n_neighbors=int(len(adata_scott_d0_sub) ** 0.5 // 4), metric='cosine')
tk.tl.triku(adata_scott_d0_sub)

sc.tl.umap(adata_scott_d0_sub, min_dist=0.5, random_state=seed)
sc.tl.leiden(adata_scott_d0_sub, resolution=3.5, random_state=seed)

assign_cats(adata_scott_d0_sub, column_groupby='id', dict_cats={'A': A_markers, 'B': B_markers}, min_score=0.4, others_name='-', key_added='krano_type')

sc.pl.umap(adata_scott_d0_sub, color=['leiden', 'n_counts', 'cell_type', 'krano_type'], legend_loc='on data')

In [ ]:
adata_scott_d0_sub.write_h5ad('data/processed/scott_d0_sub.h5')
adata_scott_d0.write_h5ad('data/processed/scott_d0.h5')

## De Micheli mouse

In [ ]:
adata_de_micheli_mouse_d0 = sc.read('data/processed/de_micheli_mouse_d0.h5')

In [ ]:
adata_de_micheli_mouse_d0.obs['id'] = adata_de_micheli_mouse_d0.obs_names
assign_cats(adata_de_micheli_mouse_d0, column_groupby='id', dict_cats={'A': A_markers, 'B': B_markers}, min_score=0.55, 
            others_name='-', key_added='krano_type')
adata_de_micheli_mouse_d0.uns['krano_type_colors'] = list_colors

In [ ]:
sc.pl.umap(adata_de_micheli_mouse_d0, color=['krano_type', 'cell_type'] , cmap=magma, alpha=0.6)

In [ ]:
sc.pl.umap(adata_de_micheli_mouse_d0, color=['krano_type', 'cell_type'] + [i for i in A_markers if i in adata_de_micheli_mouse_d0.var_names], 
           cmap=magma, alpha=0.6)

In [ ]:
sc.pl.umap(adata_de_micheli_mouse_d0, color=['krano_type', 'cell_type'] + [i for i in B_markers if i in adata_de_micheli_mouse_d0.var_names], 
           cmap=magma, alpha=0.6)

## Subsample of populations

In [ ]:
adata_de_micheli_mouse_d0_sub = adata_de_micheli_mouse_d0[adata_de_micheli_mouse_d0.obs['cell_type'].isin(['FAP Lum+', 'FAP Prg4+', 'Endothelial', 'Perivascular', 'Tenocyte', 'Neural cell', 'Glial cell'])]

sc.pp.filter_genes(adata_de_micheli_mouse_d0_sub, min_counts=1)

sc.pp.pca(adata_de_micheli_mouse_d0_sub, random_state=seed, n_comps=30)
sce.pp.bbknn(adata_de_micheli_mouse_d0_sub, metric='angular', neighbors_within_batch=3)
tk.tl.triku(adata_de_micheli_mouse_d0_sub)

sc.tl.umap(adata_de_micheli_mouse_d0_sub, min_dist=0.7, random_state=seed)
sc.tl.leiden(adata_de_micheli_mouse_d0_sub, resolution=3.5, random_state=seed)

assign_cats(adata_de_micheli_mouse_d0_sub, column_groupby='id', dict_cats={'A': A_markers, 'B': B_markers}, min_score=0.5, others_name='-', key_added='krano_type')

sc.pl.umap(adata_de_micheli_mouse_d0_sub, color=['leiden', 'cell_type', 'krano_type'], legend_loc='on data')

In [ ]:
adata_de_micheli_mouse_d0_sub.write_h5ad('data/processed/de_micheli_mouse_d0_sub.h5')
adata_de_micheli_mouse_d0.write_h5ad('data/processed/de_micheli_mouse_d0.h5')

## Giordani

In [ ]:
adata_giordani = sc.read('data/processed/giordani.h5')

In [ ]:
adata_giordani.obs['id'] = adata_giordani.obs_names
assign_cats(adata_giordani, column_groupby='id', 
            dict_cats={'A': A_markers, 'B': B_markers}, min_score=0.35, 
            others_name='-', key_added='krano_type')
adata_giordani.uns['krano_type_colors'] = list_colors

In [ ]:
sc.pl.umap(adata_giordani, color=['krano_type', 'cell_type'] , cmap=magma, alpha=0.6)

In [ ]:
sc.pl.umap(adata_giordani, color=['krano_type', 'cell_type'] + [i for i in A_markers if i in adata_giordani.var_names], 
           cmap=magma, alpha=0.6)

In [ ]:
sc.pl.umap(adata_giordani, color=['krano_type', 'cell_type'] + [i for i in B_markers if i in adata_giordani.var_names], 
           cmap=magma, alpha=0.6)

## Subsample of populations

In [ ]:
adata_giordani_sub = adata_giordani[adata_giordani.obs['cell_type'].isin(['FAP Lum+', 'FAP Prg4+', 'Endothelial', 'Perivascular', 'Tenocyte', 'Neural cell', 'Glial cell'])]

sc.pp.filter_genes(adata_giordani_sub, min_counts=1)

sc.pp.pca(adata_giordani_sub, random_state=seed, n_comps=30)
sce.pp.bbknn(adata_giordani_sub, metric='angular', neighbors_within_batch=3)
tk.tl.triku(adata_giordani_sub)

sc.tl.umap(adata_giordani_sub, min_dist=0.5, random_state=seed)
sc.tl.leiden(adata_giordani_sub, resolution=3.5, random_state=seed)

assign_cats(adata_giordani_sub, column_groupby='id', 
            dict_cats={'A': A_markers, 'B': B_markers}, min_score=0.35, 
            others_name='-', key_added='krano_type')

sc.pl.umap(adata_giordani_sub, color=['leiden', 'cell_type', 'krano_type'], legend_loc='on data')

In [ ]:
adata_giordani_sub.write_h5ad('data/processed/giordani_sub.h5')
adata_giordani.write_h5ad('data/processed/giordani.h5')

## Proietti

In [ ]:
adata_proietti = sc.read('data/processed/proietti.h5')

In [ ]:
adata_proietti.obs['id'] = adata_proietti.obs_names
assign_cats(adata_proietti, column_groupby='id', dict_cats={'A': A_markers, 'B': B_markers}, min_score=0.4, 
            others_name='-', key_added='krano_type')
adata_proietti.uns['krano_type_colors'] = list_colors

In [ ]:
sc.pl.umap(adata_proietti, color=['krano_type', 'cell_type'] , cmap=magma, alpha=0.6)

In [ ]:
sc.pl.umap(adata_proietti, color=['krano_type', 'cell_type'] + [i for i in A_markers if i in adata_proietti.var_names], 
           cmap=magma, alpha=0.6)

In [ ]:
sc.pl.umap(adata_proietti, color=['krano_type', 'cell_type'] + [i for i in B_markers if i in adata_proietti.var_names], 
           cmap=magma, alpha=0.6)

## Subsample of populations

In [ ]:
adata_proietti_sub = adata_proietti[adata_proietti.obs['cell_type'].isin(['FAP Lum+', 'FAP Prg4+', 'Endothelial', 'Perivascular', 'Tenocyte', 'Neural cell', 'Glial cell'])]

sc.pp.filter_genes(adata_proietti_sub, min_counts=1)

sc.pp.pca(adata_proietti_sub, random_state=seed, n_comps=30)
sce.pp.bbknn(adata_proietti_sub, metric='angular', neighbors_within_batch=3, batch_key='condition')
tk.tl.triku(adata_proietti_sub)

sc.tl.umap(adata_proietti_sub, min_dist=0.5, random_state=seed)
sc.tl.leiden(adata_proietti_sub, resolution=3.5, random_state=seed)

assign_cats(adata_proietti_sub, column_groupby='id', dict_cats={'A': A_markers, 'B': B_markers}, min_score=0.45, others_name='-', key_added='krano_type')

sc.pl.umap(adata_proietti_sub, color=['leiden', 'cell_type', 'krano_type'], legend_loc='on data')

In [ ]:
adata_proietti_sub.write_h5ad('data/processed/proietti_sub.h5')
adata_proietti.write_h5ad('data/processed/proietti.h5')

## Ronzoni

In [ ]:
adata_ronzoni = sc.read('data/processed/ronzoni.h5')

In [ ]:
adata_ronzoni.obs['id'] = adata_ronzoni.obs_names
assign_cats(adata_ronzoni, column_groupby='id', dict_cats={'A': A_markers, 'B': B_markers}, min_score=0.3, 
            others_name='-', key_added='krano_type')
adata_ronzoni.uns['krano_type_colors'] = list_colors

In [ ]:
sc.pl.umap(adata_ronzoni, color=['krano_type', 'cell_type'] , cmap=magma, alpha=0.6)

In [ ]:
sc.pl.umap(adata_ronzoni, color=['krano_type', 'cell_type'] + [i for i in A_markers if i in adata_ronzoni.var_names], 
           cmap=magma, alpha=0.6)

In [ ]:
sc.pl.umap(adata_ronzoni, color=['krano_type', 'cell_type'] + [i for i in B_markers if i in adata_ronzoni.var_names], 
           cmap=magma, alpha=0.6)

## Subsample of populations

In [ ]:
adata_ronzoni_sub = adata_ronzoni[adata_ronzoni.obs['cell_type'].isin(['FAP Lum+', 'FAP Prg4+', 'Endothelial', 'Perivascular', 'Tenocyte', 'Neural cell', 'Glial cell'])]

sc.pp.filter_genes(adata_ronzoni_sub, min_counts=1)

sc.pp.pca(adata_ronzoni_sub, random_state=seed, n_comps=30)
sc.pp.neighbors(adata_ronzoni_sub, random_state=seed, n_neighbors=int(len(adata_ronzoni_sub) ** 0.5 // 4), metric='cosine')
tk.tl.triku(adata_ronzoni_sub)

sc.tl.umap(adata_ronzoni_sub, min_dist=0.5, random_state=seed)
sc.tl.leiden(adata_ronzoni_sub, resolution=3.5, random_state=seed)

assign_cats(adata_ronzoni_sub, column_groupby='id', 
            dict_cats={'A': A_markers, 'B': B_markers}, 
            min_score=0.25, others_name='-', key_added='krano_type')

sc.pl.umap(adata_ronzoni_sub, color=['leiden', 'cell_type', 'krano_type'], legend_loc='on data')

In [ ]:
adata_ronzoni_sub.write_h5ad('data/processed/ronzoni_sub.h5')
adata_ronzoni.write_h5ad('data/processed/ronzoni.h5')

## Dell'Orso

In [ ]:
adata_dellorso = sc.read('data/processed/dellorso.h5')

In [ ]:
adata_dellorso.obs['id'] = adata_dellorso.obs_names
assign_cats(adata_dellorso, column_groupby='id', dict_cats={'A': A_markers, 'B': B_markers}, min_score=0.4, 
            others_name='-', key_added='krano_type')
adata_dellorso.uns['krano_type_colors'] = list_colors

In [ ]:
sc.pl.umap(adata_dellorso, color=['krano_type', 'cell_type'] , cmap=magma, alpha=0.6)

In [ ]:
sc.pl.umap(adata_dellorso, color=['krano_type', 'cell_type'] + [i for i in A_markers if i in adata_dellorso.var_names], 
           cmap=magma, alpha=0.6)

In [ ]:
sc.pl.umap(adata_dellorso, color=['krano_type', 'cell_type'] + [i for i in B_markers if i in adata_dellorso.var_names], 
           cmap=magma, alpha=0.6)

## Subsample of populations

In [ ]:
adata_dellorso_sub = adata_dellorso[adata_dellorso.obs['cell_type'].isin(['FAP Lum+', 'FAP Prg4+', 'Epcam+', 'Endothelial', 'Perivascular', 'Tenocyte', 'Neural cell', 'Glial cell'])]

sc.pp.filter_genes(adata_dellorso_sub, min_counts=1)

sc.pp.pca(adata_dellorso_sub, random_state=seed, n_comps=30)
sce.pp.bbknn(adata_dellorso_sub, metric='angular', neighbors_within_batch=3)
tk.tl.triku(adata_dellorso_sub)

sc.tl.umap(adata_dellorso_sub, min_dist=0.8, random_state=seed)
sc.tl.leiden(adata_dellorso_sub, resolution=3.5, random_state=seed)

assign_cats(adata_dellorso_sub, column_groupby='id', dict_cats={'A': A_markers, 'B': B_markers}, min_score=0.2, others_name='-', key_added='krano_type')

sc.pl.umap(adata_dellorso_sub, color=['leiden', 'cell_type', 'krano_type'], legend_loc='on data')

In [ ]:
adata_dellorso_sub.write_h5ad('data/processed/dellorso_sub.h5')
adata_dellorso.write_h5ad('data/processed/dellorso.h5')

# Running datasets against markers

## Kumar et al 2017 PC1 and PC2

In [ ]:
list_genes = ['krano_type'] + ['Rgs5', 'Acta2', 'Cxcl1', 'Cxcl2', 'Cxcl5', 'Il6', 'Il1b']

fig, axs = plt.subplots(len(list_genes), 3, figsize=(18, 4 * len(list_genes)))

for idx, gene in enumerate(list_genes):
    try:
        sc.pl.umap(adata_de_micheli_mouse_d0, color=gene, cmap=magma, ax=axs[idx][0], show=False, legend_loc='on data')
        sc.pl.umap(adata_oprescu_d0, color=gene, cmap=magma, ax=axs[idx][1], show=False, legend_loc='on data')
        sc.pl.umap(adata_scott_d0, color=gene, cmap=magma, ax=axs[idx][2], show=False, legend_loc='on data')
    except:
        pass

## Kumar et al 2017 Capillary proinflammatory/capillary and contractile/arteriolar PCs

In [ ]:
list_genes = ['krano_type'] + ['Cd274', 'Dlk1', 'Nt5e'] # Cd73 = Nt5e

fig, axs = plt.subplots(len(list_genes), 3, figsize=(18, 4 * len(list_genes)))

for idx, gene in enumerate(list_genes):
    try:
        sc.pl.umap(adata_de_micheli_mouse_d0, color=gene, cmap=magma, ax=axs[idx][0], show=False, legend_loc='on data')
        sc.pl.umap(adata_oprescu_d0, color=gene, cmap=magma, ax=axs[idx][1], show=False, legend_loc='on data')
        sc.pl.umap(adata_scott_d0, color=gene, cmap=magma, ax=axs[idx][2], show=False, legend_loc='on data')
    except:
        pass

## Birbrair PCs

In [ ]:
list_genes = ['krano_type'] + ['Pdgfrb', 'Mcam', 'Cspg4', 'Nes'] # Cd146 = Mcam, Ng2 = Cspg4

fig, axs = plt.subplots(len(list_genes), 3, figsize=(18, 4 * len(list_genes)))

for idx, gene in enumerate(list_genes):
    try:
        sc.pl.umap(adata_de_micheli_mouse_d0, color=gene, cmap=magma, ax=axs[idx][0], show=False, legend_loc='on data')
        sc.pl.umap(adata_oprescu_d0, color=gene, cmap=magma, ax=axs[idx][1], show=False, legend_loc='on data')
        sc.pl.umap(adata_scott_d0, color=gene, cmap=magma, ax=axs[idx][2], show=False, legend_loc='on data')
    except:
        pass

## Camps ISC

* ISC1: Ly6c1, Cd55
* ISC2: Gdf10, Meox2, F3/Cd142
* ISC3: Thbs4, Fbln7, Sdc1

In [ ]:
list_genes = ['krano_type'] + ['Cd55', 'F3', 'Sdc1']

fig, axs = plt.subplots(len(list_genes), 3, figsize=(18, 4 * len(list_genes)))

for idx, gene in enumerate(list_genes):
    try:
        sc.pl.umap(adata_de_micheli_mouse_d0, color=gene, cmap=magma, ax=axs[idx][0], show=False, legend_loc='on data')
        sc.pl.umap(adata_oprescu_d0, color=gene, cmap=magma, ax=axs[idx][1], show=False, legend_loc='on data')
        sc.pl.umap(adata_scott_d0, color=gene, cmap=magma, ax=axs[idx][2], show=False, legend_loc='on data')
    except:
        pass

# Beautiful figs

In [ ]:
fig_dir = 'figures/'
if not os.path.exists(fig_dir + 'clusters/'): os.makedirs(fig_dir + 'clusters/')

In [ ]:
# MPL config
font = {'family' : 'normal',
        'weight' : 'light',
        'size'   : 15}

# mpl.rc('font', **font)

In [ ]:
def makefig(list_genes, name_order=None, adata_list=[adata_oprescu_d0, adata_scott_d0, adata_de_micheli_mouse_d0], 
           list_datasets = ['Oprescu', 'Scott', 'De Micheli']):
    n_cols = len(adata_list)
    fig, axs = plt.subplots(len(list_genes), n_cols, figsize=(6 * n_cols, 4 * len(list_genes)))
    
        
    for idx, gene in enumerate(list_genes):
        try:
            for adata_idx, adata in enumerate(adata_list):
                sc.pl.umap(adata, color=gene, cmap=magma, ax=axs[idx][adata_idx], show=False, legend_fontsize=11)
        except:
            raise
               
    for ax_idx, ax in enumerate(axs.ravel()):
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        ax.spines['bottom'].set_visible(False)
        if ax_idx % n_cols == 0:
            ax.spines['left'].set_visible(False)
            
        ax.set_xlabel('')
        
        if ax_idx % n_cols == 0:
            ax.set_ylabel(list_genes[ax_idx // n_cols], fontsize=20)
        else:
            ax.set_ylabel('')
            
        if ax_idx in range(len(list_datasets)):
            ax.set_title(list_datasets[ax_idx], fontsize=20)
        else:
            ax.set_title('')
            
        # legend unification 
        if ax_idx % n_cols == 0:
            dict_legends = {}
        
        try:
            legend = ax.get_legend()
            names, handles = [str(x._text) for x in legend.texts], legend.legendHandles  
            for i in range(len(names)):
                handle = handles[i]
                handle.set_sizes([80])
                dict_legends[names[i]] = handles[i]
            if ax_idx % n_cols != n_cols - 1:
                ax.get_legend().remove()
            else:
                if name_order is None:
                    ax.legend(dict_legends.values(), dict_legends.keys(), bbox_to_anchor=(1.05, 0.8), frameon=False, prop={'size': 19})
                else:
                    ax.legend([dict_legends[i] for i in name_order if i in dict_legends.keys()], 
                              [i for i in name_order if i in dict_legends.keys()], bbox_to_anchor=(1.05, 1.01), frameon=False, prop={'size': 19})
        except:
            pass 
        
    
    plt.tight_layout()
    plt.savefig(fig_dir + 'clusters/' + '-'.join(list_genes) + '.png', dpi=500)
    plt.savefig(fig_dir + 'clusters/' + '-'.join(list_genes) + '.pdf')
    


## Fig 2I & Sup 1

In [ ]:
name_order = ['Endothelial', 'Pericyte', 'Fibroblast', 'FAP Lum+', 'FAP Prg4+', 'Tenocyte', 'Neural cell', 'Glial cell',
                                                          'Myonuclei', 'MuSC', 'Immune', 'APC / Proliferative ICs', 
                                                          'Monocyte', 'Neutrophil', 'Myeloid', 'B cell', 'T cell', 'A', 'B', 'C', 'Other']

name_order_dict = {'Endothelial': '#950541', 'FAP Lum+': '#f46d43', 'FAP Prg4+': '#fdae61', 'Glial cell': '#3b8957',  'Neural cell': '#3898d3',  'Tenocyte': '#66c2a5'}




adata_oprescu_d0_sub.uns['cell_type_colors'] = [name_order_dict[i] for i in adata_oprescu_d0_sub.obs['cell_type'].cat.categories]
adata_scott_d0_sub.uns['cell_type_colors'] = [name_order_dict[i] for i in adata_scott_d0_sub.obs['cell_type'].cat.categories]
adata_de_micheli_mouse_d0_sub.uns['cell_type_colors'] = [name_order_dict[i] for i in adata_de_micheli_mouse_d0_sub.obs['cell_type'].cat.categories]


# Add colors about krano
adata_oprescu_d0_sub.obs['is_krano'] = ['kranocyte' if i in ['A'] else '-' for i in adata_oprescu_d0_sub.obs['krano_type']]
adata_scott_d0_sub.obs['is_krano'] = ['kranocyte' if i in ['A'] else '-' for i in adata_scott_d0_sub.obs['krano_type']]
adata_de_micheli_mouse_d0_sub.obs['is_krano'] = ['kranocyte' if i in ['A'] else '-' for i in adata_de_micheli_mouse_d0_sub.obs['krano_type']]

adata_oprescu_d0_sub.uns['is_krano_colors'] = ['#cdcdcd', '#454545', ]
adata_scott_d0_sub.uns['is_krano_colors'] = ['#cdcdcd', '#454545', ]
adata_de_micheli_mouse_d0_sub.uns['is_krano_colors'] = ['#cdcdcd', '#454545', ]

In [ ]:
makefig(['cell_type', 'is_krano'], name_order=name_order, adata_list=[adata_oprescu_d0_sub, adata_scott_d0_sub, adata_de_micheli_mouse_d0_sub], )

In [ ]:
makefig(['cell_type', 'Cd34', 'Ngfr', 'Pdgfrb'], name_order=name_order, adata_list=[adata_oprescu_d0_sub, adata_scott_d0_sub, adata_de_micheli_mouse_d0_sub], )

## Fig Sup Tnc

In [ ]:
name_order_dict = {'APC': '#9782f2', 'Endothelial': '#950541', 'FAP Lum+': '#f46d43', 'FAP Prg4+': '#fdae61', 'Immune': '#5645a0', 'Macrophage': '#2576a8', 'Monocyte': '#409aef', 
                   'Myonuclei': '#d5ee53', 'Neutrophil': '#7ce7f4', 'Pericyte': '#f4106f', 'Satellite cell': '#abdda4', 'T cell': '#c1e8e7', 'Tenocyte': '#66c2a5', }


adata_oprescu_d0.uns['cell_type_colors'] = [name_order_dict[i] for i in adata_oprescu_d0.obs['cell_type'].cat.categories]
adata_oprescu_d2.uns['cell_type_colors'] = [name_order_dict[i] for i in adata_oprescu_d2.obs['cell_type'].cat.categories]
adata_oprescu_d35.uns['cell_type_colors'] = [name_order_dict[i] for i in adata_oprescu_d35.obs['cell_type'].cat.categories]
adata_oprescu_d5.uns['cell_type_colors'] = [name_order_dict[i] for i in adata_oprescu_d5.obs['cell_type'].cat.categories]
adata_oprescu_d10.uns['cell_type_colors'] = [name_order_dict[i] for i in adata_oprescu_d10.obs['cell_type'].cat.categories]
adata_oprescu_d21.uns['cell_type_colors'] = [name_order_dict[i] for i in adata_oprescu_d21.obs['cell_type'].cat.categories]


In [ ]:
makefig(['cell_type', 'Tnc'], name_order=name_order, 
        adata_list=[adata_oprescu_d0, adata_oprescu_d2, adata_oprescu_d5, adata_oprescu_d10, adata_oprescu_d21], 
        list_datasets=['D0', 'D2', 'D5', 'D10', 'D21'])